In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine

In [2]:
server = r"DESKTOP-5VCHN8F\NODESERVER"
database = "Sales Analysis"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
)

In [3]:
os.makedirs("visuals", exist_ok=True)

In [4]:
query_rfm = """
with CustomerRFM as ( 
    select City + ', '+State as CustomerLocation,
           Segment,
           Datediff(DAY,MAX(Order_Date),GETDATE()) as Recency,
           count(distinct Order_Id) as Frequency,
           SUM(List_price * Quantity * (1-(Discount_Percent/100))) as Monetary,
           ntile(5) over (order by Datediff(day,max(Order_Date),getdate())desc) as R_Score,
           ntile(5) over (order by count(distinct Order_Id)) as F_Score,
           ntile(5) over (order by SUM(List_price * Quantity * (1-(Discount_Percent/100)))) as M_Score,
           (ntile(5) over (order by datediff(day,max(Order_Date),Getdate()) desc) 
            + ntile(5) over (order by count(distinct Order_Id))
            + ntile(5) over (order by SUM(List_price * Quantity * (1-(Discount_Percent/100))))) / 3.0 as RFM_Score
    from Order_Sales group by City , State , Segment
)
select CustomerLocation,Segment,Recency,Frequency,Monetary,
       R_Score,F_Score,M_Score,RFM_Score,
       case
         when RFM_Score >= 4.5 then 'Champions'
         when RFM_Score >= 4.0 then 'Loyal Customers'
         when RFM_Score >= 3.5 then 'Potenial Loyalits'
         when RFM_Score >= 3.0 then 'New Customers'
         when RFM_Score >= 2.5 then 'Promising'
         when RFM_Score >= 2.0 then 'Needs Attention'
         else 'At Risk'
       end as CustomerSegment
from CustomerRFM order by RFM_Score desc;
"""
df_rfm = pd.read_sql(query_rfm, engine)

In [5]:
# Pie chart - Segments
segment_counts = df_rfm["CustomerSegment"].value_counts()
plt.pie(segment_counts, labels=segment_counts.index, autopct="%1.1f%%", startangle=90)
plt.title("Customer Segments")
plt.savefig("visuals/customer_segments.png")
plt.close()

In [6]:
# Histogram - RFM Score
plt.hist(df_rfm["RFM_Score"], bins=10, color="skyblue", edgecolor="black")
plt.title("RFM Score Distribution")
plt.xlabel("RFM Score"); plt.ylabel("Customers")
plt.savefig("visuals/rfm_score_distribution.png")
plt.close()

In [7]:
# Avg R,F,M by Segment
avg_rfm = df_rfm.groupby("CustomerSegment")[["Recency","Frequency","Monetary"]].mean()
avg_rfm.plot(kind="bar", figsize=(10,6))
plt.title("Average Recency, Frequency, Monetary by Segment")
plt.ylabel("Average Value")
plt.savefig("visuals/avg_rfm_by_segment.png")
plt.close()